### 동적 배열(Dynamic array)  

스택 영역에 배열을 만들기 위해서는 고정된 크기로 만들어야 한다.  
이렇게 만들어지는 배열은 가변적인 상황에 사용하기 어렵다.  
그래서 힙 영역에 동적 배열을 만들게 됐다.  

파이썬 배열도 동적 배열인데 원소의 크기나 배열의 길의 변화가 유연하다.  

[ 높은 주소 ]  


| 메모리 공간       | 비고                   | 예(python)                |
| ------------ | -------------------- | ------------------------ |
| Stack        | ← 함수 호출마다 push/pop   | 지역 스코프의 변수               |
| Heap         | ← 동적 메모리, 위로 증가      | 동적인 자료 구조들(list, dict 등) |
| BSS Segment  | ← 전역/정적 변수 (초기값 X)   | 전역 스코프의 초기화면 된 변수        |
| Data Segment | ← 전역/정적 변수 (초기값 O)   | 전역 스코프의 변수 선언까지 된 변수     |
| Text Segment | ← 실행 코드              | 코드 자체                    |


[ 낮은 주소 ]    

### 지역성의 원리와 캐시(Principle of locality and Cache)  

배열의 특징  
메모리상에서 물리적, 선형적으로 이어져 있다  
해당 장점이 지역성의 원리와 캐시에서 이용된다  

In [ ]:
# principle of locality

def sum_all(arr):
    ret = 0 # 1
    for elem in arr: #2
        ret+=elem    #3
    return ret

#### Cache 사용 이유  

CPU가 계산하는 절차를 위의 코드로 예를 들면  

사용 부품  
    CPU  
    CPU의 레지스터  
    CPU 연산 장치 ALU(Arithmetic Logic Unit)  
    메인 메모리  

절차  
    1. 메인 메모리에서 ret을 레지스터로 가져온다 (20 ~100클럭)   
    2. 메인 메모리에서 elem을 레지스터로 가져온다 (20 ~ 100클럭)  
    3. ret,elem을 ALU로 전달 (1 클럭)  
    4. ALU 계산 후 레지스터로 전달(1 클럭)  
    5. 레지스터에 있는 계산 결과 메인 메모리로 전달 (20 ~ 100클럭)  
    6. 해당 작업을 배열 개수 만큼 진행  

1 회 loop마다 최소 60클럭을 소요를 모든 배열에 진행하는 건 매우 비효율적  

그래서 캐시를 도입한다.  

캐시의 위치는 레지스터와 메인 메모리 사이  

절차  
    1. CPU가 캐시에 데이터 요청  
    2-1. 없으면(chache miss) 메인 메모리에서 데이터를 캐시로 가져온다 (20 ~ 100클럭)  
         이 때 요청했던 데이터만 가져오는 게 아니라 전략적으로 데이터를 추가한다.  
    2-2. 있으면(cache hit) 캐시에 있는 데이터를 레지스터로 가져온다 (3 클럭)  
    3. 데이터를 ALU로 전달 (1 클럭)  
    4. ALU 계산 후 레지스터로 전달 (1 클럭)  
    5. 계산 결과 cache로 전달 (3 클럭)  
    6. loop가 끝나기 전 까지 중간 계산 결과는 cache에 유지(write-back)
    7. loop가 끝나기 전에는 cache에, 끝나면 결과 데이터를 메인 메모리로 전달 (20 ~ 100클럭)  

캐시가 있을 때 자원을 매우 효율적으로 사용 할 수 있다.  

전략  
공간 지역성(spatial locality) : CPU에서 요청한 데이터와 가까운 데이터를 모두 캐시로 전달  
프리페치 : CPU가 아직 요청하지 않는 데이터도 미리 가져옴  
캐시 라인 : 데이터를 주변까지 포함해 묶음 단위로 캐시에 저장  

가져오는 전략이 있는 것 처럼 교체하는 전략도 있다.  

LRU(Leash Recently Used) : 가장 오래 전에 쓰인 데이터 제거  
LFU(Leash Frequently Used) : 사용 횟수가 적은 데이터 제거  
Random : 무작위 제거  

이런 전략들을 이용해서 ceche hit 비율을 높일수록 효율이 증가한다.  

배열은 지역성의 원리와 캐시를 활용하기 좋은 데이터 묶음  


### 인덱싱  

데이터 주소 값 = 배열의 첫 주소 값 + (데이터 크기*인덱스)  



### 동적 배열에서 데이터 삽입과 삭제 1  

배열 마지막에 데이터를 추가하거나 배열 마지막 데이터를 제거 할 때 시간 복잡도 O(1)  

배열 생성 시 확보하는 메모리 공간을 capacity라고 하고 채워진 데이터 크기를 size라고 한다.  
capacity가 가득 차있지 않을 때는 O(1)로 데이터를 추가 할 수 있다.  
capacity가 가득 차있는 경우는?  
가득 차면 기존 배열의 두 배 크기로 capacity를 다시 잡는데 이 때 딱 한 번 O(n)이 된다.  
이후 계속 O(1)인데 이를 분할 상환 분석(amortized analysis)라고 한다.  
한 번 O(n)을 이전 나머지 연산들이 분할해서 비용을 지불했다고 생각하는 것이다.  
조금 자세히 보면 O(1.xxx)가 되겠지만 O(1)로 본다.  

### 동적 배열에서 데이터 삽입과 삭제 2  

배열 중간에 데이터를 삽입하거나 삭제하는 연산은 O(n)  

삽입은 삽입 할 위치 이후 데이터를 오른쪽으로 한 칸씩 밀어야 하고,  
삭제는 삭제한 데이터 위치 이후 데이터를 왼쪽으로 한 칸씩 밀어야 한다.  